In [2]:
from PIL import Image
from sklearn.cluster import KMeans
import numpy as np

In [3]:
# read images from local directory

im01 = np.array(Image.open('./RobertMixed03.jpg'))
im02 = np.array(Image.open('./smallstrelitzia.jpg'))
im03 = np.array(Image.open('./smallsunset.jpg'))
pixs01 = np.reshape(im01, (im01.shape[0]*im01.shape[1], 3))
pixs02 = np.reshape(im02, (im02.shape[0]*im02.shape[1], 3))
pixs03 = np.reshape(im03, (im03.shape[0]*im03.shape[1], 3))

In [4]:
# initialize parameters

    # Inputs: pixels: RGB image pixels
    #         segments: number of clusters
    #
    # Return: mu: cluster centers
    #         pi: cluster weights

def init_para(pixels, segments):
    
    kmeans = KMeans(n_clusters = segments, init = "random", random_state = 0).fit(pixels)
    mu = kmeans.cluster_centers_
    pi = np.ones((segments, 1))/segments
    
    return mu, pi

In [6]:
# E-step

    # Inputs: pixels: RGB image pixels in (r,g,b) form
    #         mu: cluster centers, size: (segments * 3)
    #         pi: cluster weights, size: (segments * 1)
    #
    # Return: w: weights that a data belongs to a cluster
    
def estimation(pixels, mu, pi):
    
    w = np.zeros((pixels.shape[0], mu.shape[0]))
    upper = np.zeros((pixels.shape[0], mu.shape[0]))
    bottom = np.zeros((pixels.shape[0], mu.shape[0]))
    
    for j in range(mu.shape[0]):
        diff = pixels - mu[j]
        square = np.sum(np.abs(diff)**2,axis=-1)
        upper[:,j] = np.exp(-0.5 * square)*pi[j]
        
        for i in range(mu.shape[0]):
            pixel = pixels - mu[i]
            square_temp = np.sum(np.abs(pixel)**2, axis = -1)
            pixel = np.exp(-0.5 * square_temp)*pi[i]
            pixel = np.reshape(np.array(pixel), (pixels.shape[0],1))
            bottom[:,j] = bottom[:,j] + pixel
    w = upper/bottom
        
    return w

In [ ]:
w = estimation(pixs01, mu0, pi0)


In [ ]:
# E-step

    # Inputs: pixels: RGB image pixels in (r,g,b) form
    #         mu: cluster centers, size: (segments * 3)
    #         pi: cluster weights, size: (segments * 1)
    #
    # Return: w: weights that a data belongs to a cluster
    
def estimation(pixels, mu, pi):
    
    w = np.zeros((pixels.shape[0], mu.shape[0]))
    for x in range(pixels.shape[0]):
        pixel = np.array([pixels[x]]*mu.shape[0])
        diff = np.subtract(pixel, mu)
        square = np.multiply(diff, diff)
        min_idx = np.argmin(np.sum(square, axis = 1))
        dmin = square[min_idx]
        square = np.subtract(square, dmin)
        power = -1/2 * np.sum(square, axis=1)
        weight = np.multiply(np.exp(power), pi.T)
        w[x] = np.divide(weight, np.sum(weight))
        
    return w

In [ ]:
# M-step

    # Inputs: pixels: RGB image pixels
    #         w: weights that a data belongs to a cluster  
    #
    # Return: mu: cluster centers
    #         pi: cluster weights  
    
def maximization(pixels, w):
    
    mu = np.zeros((w.shape[1], pixels.shape[1]))
    pi = np.zeros((w.shape[1], 1))
    for c in range(w.shape[1]):
        
        w_c = w.T[c]
        mu_num = np.matmul(pixels.T, w_c)
        mu_denom = np.sum(w_c)
        mu[c] = np.divide(mu_num, mu_denom)
        pi[c] = np.divide(mu_denom, pixels.shape[0])
        
    return mu, pi

In [ ]:
# EM iteration

    # Inputs: it: iterating times
    #         pixels: RGB image pixels 
    #         mu: cluster centers
    #         pi: cluster weights  
    #
    # Return: mu: updated cluster centers after iterations
    #         pi: updated cluster weights after iterations

def EM(it, pixels, mu, pi):
    
    if it==0:
        
        return mu,pi
    
    else:
        
        w = estimation(pixels, mu, pi)
        mu, pi = maximization(pixels, w)
        
        return EM(it-1, pixels, mu, pi)

In [5]:
# image1 clustering

it = 10
segments = 10
mu0, pi0 = init_para(pixs01, segments)
#mu, pi = EM(it, pixs01, mu0, pi0)

# TODO: plot the image using pixels closest to the center of clusters
# Your code here


In [ ]:
print(mu)
print(pi)